In [64]:
import pandas as pd
arquivos = pd.read_csv('CRAM/genia.test.iob2', delimiter='\t', skip_blank_lines = False, header = None)
arquivos.columns=["token", "Label1", "Label2", "Label3", "Label4"]
print(arquivos.head())

def is_nan(x):
    return (x != x)

dicEntidades = {}
listaTags=['DNA', 'RNA','cell_type', 'cell_line','protein']

tokens = arquivos['token']
label1 = arquivos['Label1']
label2 = arquivos['Label2']
label3 = arquivos['Label3']
label4 = arquivos['Label4']

fraseCompleta=''
i=0
for token in tokens:
    #print(token)
    #print(is_nan(token))
    if token and not is_nan(token):
        fraseCompleta = fraseCompleta + ' ' + token
    else:
        dicEntidades[i] = [fraseCompleta, []]
        fraseCompleta = ""
        i=i+1    
    if i>10:
        break

#dicEntidades

listaEntidades=[]
for tag in listaTags:
    i=0
    j=0
    #print(tag)
    for token, l1 in zip(tokens, label1):
        #print(token)
        #print(l1)
        if token and not is_nan(token):
            #print('l1:', l1)
            #print('tag:', tag)
            #print('B-'+tag in l1)
            if 'B-'+tag in l1:
                if isEnt==1: # acabou uma e começou outra do mesmo tipo
                    isEnt = 0
                    listaEntidades.append([nomeEntidade,tag,indices])
                    #print('fim, nomeEntidade:', nomeEntidade)
                    nomeEntidade=''
                    indices=[]
                nomeEntidade = token
                #print('inicio, nomeEntidade:', nomeEntidade)
                isEnt = 1
                indices.append(j)
                #print(nomeEntidade)
            elif 'I-'+tag in l1:
                nomeEntidade = nomeEntidade +' '+ token
                #print('continua, nomeEntidade:', nomeEntidade)
                indices.append(j)
            else:
                #print('isDNA:', isDNA)
                if isEnt==1:
                    isEnt = 0
                    #print('DNA:', nomeEntidade)
                    listaEntidades.append([nomeEntidade,tag,indices])
                    #print('fim, nomeEntidade:', nomeEntidade)
                    nomeEntidade=''
                    indices=[]   
            j=j+1
        else:
            frase=dicEntidades[i][0]
            entidadesTava=dicEntidades[i][1]
            #print(listaEntidades)
            #print(frase)
            if len(listaEntidades)>0:
                entidadesTava.append(listaEntidades)
            dicEntidades[i] = [frase, entidadesTava]
            listaEntidades=[]
            j=0
            i=i+1    
        if i>10:
            break
       
    i=0
    j=0    
    for token, l1 in zip(tokens, label2):
        #print(token)
        #print(l1)
        if token and not is_nan(token):
            #print('l1:', l1)
            #print('tag:', tag)
            #print('B-'+tag in l1)
            if 'B-'+tag in l1:
                if isEnt==1: # acabou uma e começou outra do mesmo tipo
                    isEnt = 0
                    listaEntidades.append([nomeEntidade,tag,indices])
                    #print('fim, nomeEntidade:', nomeEntidade)
                    nomeEntidade=''
                    indices=[]
                nomeEntidade = token
                #print('inicio, nomeEntidade:', nomeEntidade)
                isEnt = 1
                indices.append(j)
                #print(nomeEntidade)
            elif 'I-'+tag in l1:
                nomeEntidade = nomeEntidade +' '+ token
                #print('continua, nomeEntidade:', nomeEntidade)
                indices.append(j)
            else:
                #print('isDNA:', isDNA)
                if isEnt==1:
                    isEnt = 0
                    #print('DNA:', nomeEntidade)
                    listaEntidades.append([nomeEntidade,tag,indices])
                    #print('fim, nomeEntidade:', nomeEntidade)
                    nomeEntidade=''
                    indices=[]   
            j=j+1
        else:
            frase=dicEntidades[i][0]
            entidadesTava=dicEntidades[i][1]
            #print(listaEntidades)
            #print(frase)
            if len(listaEntidades)>0:
                entidadesTava.append(listaEntidades)
            dicEntidades[i] = [frase, entidadesTava]
            listaEntidades=[]
            j=0
            i=i+1    
        if i>10:
            break        
            
    
#for token, l1, l2, l3, l4 in zip(tokens, label1, label2, label3, label4):
#    print(token, l1, l2, l3, l4)
#    break
dicEntidades

        token Label1 Label2 Label3 Label4
0       There      O      O      O      O
1          is      O      O      O      O
2           a      O      O      O      O
3      single      O      O      O      O
4  methionine  B-DNA      O      O      O


{0: [' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .',
  [[['methionine codon - initiated open reading frame',
     'DNA',
     [4, 5, 6, 7, 8, 9, 10]],
    ['homeobox', 'DNA', [20]],
    ['CAX repeat', 'DNA', [23, 24]],
    ['open reading frame', 'DNA', [28, 29, 30]]]]],
 1: [' When the homeodomain from HB24 was compared to known mammalian and Drosophila homeodomains it was found to be only moderately conserved , but when it was compared to a highly diverged Drosophila homeodomain , H2 . 0 , it was found to be 80 % identical .',
  [[['homeodomain', 'DNA', [2]],
    ['HB24', 'DNA', [4]],
    ['homeodomain', 'DNA', [32]]],
   [['Drosophila homeodomain', 'DNA', [31, 32]]]]],
 2: [' The HB24 mRNA was absent or present at low levels in normal B and T lymphocytes ; however , with the appropriate activation signal HB24 mRNA was induce

In [7]:
import csv
dicEntidades = {}
listaTags=['DNA', 'RNA','cell_type', 'cell_line','protein']
with open('CRAM/genia.test.iob2', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='\t', quotechar='"')
    i=0
    listaEntidades=[]
    j=0
    for row in spamreader:
        if row:
            #print(', '.join(row))
            #print(row)
            token=row[0]
            label=row[1:5]
            listaEntidades.append([token, label, str(j)])
            j=j+1
            if i>10:
                break
        else:
            listaEntidades2=[]
            nomeEntidade=''
            indices=[]
            isEnt = 0
            fraseCompleta=''
            for entidade in listaEntidades:
                fraseCompleta = fraseCompleta+' '+entidade[0]
            indiceColuna=-1
            for tag in listaTags:
                indiceColuna=indiceColuna+1
                print(indiceColuna)
                print(colunas[indiceColuna])
                coluna=colunas[indiceColuna]
                for entidade in listaEntidades:
                    colunas = entidade[1]
                    print('colunas:', colunas)
                    #print('coluna:', coluna)
                    if 'B-'+tag in colunas[indiceColuna]:
                        nomeEntidade = entidade[0]
                        print('inicio DNA, nomeEntidade:', nomeEntidade)
                        isEnt = 1
                        indices.append(entidade[2])
                        #print(nomeEntidade)
                    elif 'I-'+tag in colunas[indiceColuna]:
                        nomeEntidade = nomeEntidade +' '+ entidade[0]
                        print('continua DNA, nomeEntidade:', nomeEntidade)
                        indices.append(entidade[2])
                    else:
                        #print('isDNA:', isDNA)
                        if isEnt==1:
                            isEnt = 0
                            #print('DNA:', nomeEntidade)
                            listaEntidades2.append([nomeEntidade,tag,indices])
                            print('fim DNA, nomeEntidade:', nomeEntidade)
                            nomeEntidade=''
                            indices=[]
                    dicEntidades[i]=[fraseCompleta,listaEntidades2]
            i=i+1
            listaEntidades=[]
            j=0
        
            
dicEntidades

0
O
colunas: ['O', 'O', 'O', 'O']
colunas: ['O', 'O', 'O', 'O']
colunas: ['O', 'O', 'O', 'O']
colunas: ['O', 'O', 'O', 'O']
colunas: ['B-DNA', 'O', 'O', 'O']
inicio DNA, nomeEntidade: methionine
colunas: ['I-DNA', 'O', 'O', 'O']
continua DNA, nomeEntidade: methionine codon
colunas: ['I-DNA', 'O', 'O', 'O']
continua DNA, nomeEntidade: methionine codon -
colunas: ['I-DNA', 'O', 'O', 'O']
continua DNA, nomeEntidade: methionine codon - initiated
colunas: ['I-DNA', 'O', 'O', 'O']
continua DNA, nomeEntidade: methionine codon - initiated open
colunas: ['I-DNA', 'O', 'O', 'O']
continua DNA, nomeEntidade: methionine codon - initiated open reading
colunas: ['I-DNA', 'O', 'O', 'O']
continua DNA, nomeEntidade: methionine codon - initiated open reading frame
colunas: ['O', 'O', 'O', 'O']
fim DNA, nomeEntidade: methionine codon - initiated open reading frame
colunas: ['O', 'O', 'O', 'O']
colunas: ['O', 'O', 'O', 'O']
colunas: ['O', 'O', 'O', 'O']
colunas: ['O', 'O', 'O', 'O']
colunas: ['O', 'O', 'O'

IndexError: list index out of range

In [ ]:
import csv
dicEntidades = {}
listaTags=['DNA', 'RNA','cell_type', 'cell_line','protein']
with open('CRAM/genia.test.iob2', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='\t', quotechar='"')
    i=0
    listaEntidades=[]
    j=0
    for row in spamreader:
        if row:
            #print(', '.join(row))
            #print(row)
            token=row[0]
            label=row[1:5]
            listaEntidades.append([token, label, str(j)])
            j=j+1
            if i>10:
                break
        else:
            listaEntidades2=[]
            nomeEntidade=''
            indices=[]
            isEnt = 0
            fraseCompleta=''
            for entidade in listaEntidades:
                fraseCompleta = fraseCompleta+' '+entidade[0]
            indiceColuna=-1
            for tag in listaTags:
                indiceColuna=indiceColuna+1
                print(indiceColuna)
                print(colunas[indiceColuna])
                coluna=colunas[indiceColuna]
                for entidade in listaEntidades:
                    colunas = entidade[1]
                    print('colunas:', colunas)
                    #print('coluna:', coluna)
                    if 'B-'+tag in coluna[indiceColuna]:
                        nomeEntidade = entidade[0]
                        print('inicio DNA, nomeEntidade:', nomeEntidade)
                        isEnt = 1
                        indices.append(entidade[2])
                        #print(nomeEntidade)
                    elif 'I-'+tag in coluna[indiceColuna]:
                        nomeEntidade = nomeEntidade +' '+ entidade[0]
                        print('continua DNA, nomeEntidade:', nomeEntidade)
                        indices.append(entidade[2])
                    else:
                        #print('isDNA:', isDNA)
                        if isEnt==1:
                            isEnt = 0
                            #print('DNA:', nomeEntidade)
                            listaEntidades2.append([nomeEntidade,tag,indices])
                            print('fim DNA, nomeEntidade:', nomeEntidade)
                            nomeEntidade=''
                            indices=[]
                    dicEntidades[i]=[fraseCompleta,listaEntidades2]
            i=i+1
            listaEntidades=[]
            j=0
        
            
dicEntidades